<a href="https://colab.research.google.com/github/mn22abk/DSProjectHamza2024/blob/main/DSPHamzafileIV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
!pip install arch
from arch import arch_model
!pip install ta
import ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=6cb6e5c2b458a38ced3ec93dc88c6560109cf53e2e72be41ed5127d8c6ad7179
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [2]:
# Load the data
df = pd.read_csv('prices-split-adjusted.csv')

In [4]:
# Convert date column to datetime, handling different formats
df['date'] = pd.to_datetime(df['date'], errors='coerce')

In [7]:
# Drop rows with invalid dates
df.dropna(subset=['date'], inplace=True)


In [8]:
# Remove duplicate dates by taking the first occurrence
df = df.drop_duplicates(subset=['date'], keep='first')

In [9]:
# Ensure the date index is sorted
df.sort_values('date', inplace=True)


In [10]:
# Set date as index
df.set_index('date', inplace=True)

In [11]:
# Set frequency to business days
df = df.asfreq('B')
